In [348]:
from pandas import read_csv
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
# from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import train_test_split # splitting data
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline           # for making pipelines
# from sklearn.neighbors import KNeighborsRegressor    # regressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings('ignore')


In [352]:
#read in the en csv files 
all_data = read_csv('data/data.csv')
all_data.isnull().sum()
for col_name in all_data.columns:
    new_col_name = col_name.split(None, 1)[1:]
    new_col_name = ''.join(new_col_name)
    all_data.rename(columns = {col_name:new_col_name[:-33]}, inplace = True)
all_data.columns = all_data.columns.str.replace('-', ' ')
all_data.columns = all_data.columns.str.replace(' ', '_')

all_data = all_data[pd.notnull(all_data['Median_Listing_Price___All_Homes'])]


In [353]:
month = []
year = []
for date in all_data['']:
    month.append(date[0])
    year.append(date[-4:])

all_data['month'] = month
all_data['year'] = year
all_data = all_data.drop('', axis=1)
#all_data = all_data.drop('Zillow_Home_Value_Index_-_Top_Tier_-_Year-Over...', axis=1)

all_data['month'] = all_data['month'].astype('int64')
all_data['year']=all_data['year'].astype('int64')

col_name = all_data.columns


In [354]:
from sklearn.impute import SimpleImputer
impu = SimpleImputer()
all_data = pd.DataFrame(impu.fit_transform(all_data))
all_data.columns = col_name


In [355]:
# #standardized

# from sklearn import preprocessing
# # Get column names first
# names = all_data.columns
# # Create the Scaler object
# std_scaler = preprocessing.StandardScaler()
# # Fit your data on the scaler object
# scaled_data = std_scaler.fit_transform(all_data)
# scaled_data = pd.DataFrame(scaled_data, columns=names)

#normalize
from sklearn import preprocessing
# Normalize total_bedrooms column
names = all_data.columns
scaled_data = preprocessing.normalize(all_data)
scaled_data = pd.DataFrame(scaled_data, columns=names)
scaled_data = scaled_data[(scaled_data > 0).all(1)]
scaled_data

Home_Sold_As_Foreclosure___Ratio___All_Homes  Inventory_Measure_(Public)  \
31                                   1.981104e-06                    0.000797   
32                                   2.378582e-06                    0.000828   
33                                   2.225272e-06                    0.000764   
34                                   2.562393e-06                    0.000653   
35                                   3.879104e-06                    0.000489   
36                                   4.466468e-06                    0.000436   
37                                   4.855642e-06                    0.000493   
38                                   4.746512e-06                    0.000536   
39                                   4.741808e-06                    0.000528   
40                                   4.214962e-06                    0.000585   
41                                   3.715427e-06                    0.000638   
42                                   4.163156e-06                    0.000671   
43                                   4.045787e-06                    0.000698   
44                                   3.517415e-06                    0.000707   
45                                   3.852482e-06                    0.000663   
46                                   3.483534e-06                    0.000618   
47                                   2.772032e-06                    0.000501   
48                                   3.066235e-06                    0.000437   
49                                   3.209579e-06                    0.000427   
50                                   3.336812e-06                    0.000459   
51                                   3.115631e-06                    0.000507   
52                                   2.797157e-06                    0.000571   
53                                   2.655337e-06                    0.000595   
54                                   2.396925e-06                    0.000610   
55                                   2.453996e-06                    0.000610   
56                                   2.288649e-06                    0.000657   
57                                   2.202551e-06                    0.000607   
58                                   2.312751e-06                    0.000494   
59                                   2.341536e-06                    0.000389   
60                                   2.214108e-06                    0.000312   
..                                            ...                         ...   
77                                   4.738977e-07                    0.000356   
78                                   4.375557e-07                    0.000374   
79                                   4.384003e-07                    0.000384   
80                                   4.019822e-07                    0.000406   
81                                   4.249018e-07                    0.000353   
82                                   3.339782e-07                    0.000275   
83                                   3.390111e-07                    0.000188   
84                                   3.728506e-07                    0.000161   
85                                   4.301454e-07                    0.000160   
86                                   3.134857e-07                    0.000169   
87                                   2.121424e-07                    0.000183   
88                                   1.281426e-07                    0.000203   
89                                   1.805197e-07                    0.000235   
90                                   1.576989e-07                    0.000240   
91                                   1.585928e-07                    0.000207   
92                                   1.544511e-06                    0.000237   
93                                   1.491161e-06                    0.000232   
94                                  

In [356]:
#Removing features with low variance
def variance_threshold_selector(data):
    sel = VarianceThreshold()
    sel.fit(data)
    return data[data.columns[sel.get_support(indices=True)]]


var_thrhold = variance_threshold_selector(scaled_data) 


scaled_data.columns

Index(['Home_Sold_As_Foreclosure___Ratio___All_Homes',
       'Inventory_Measure_(Public)', 'Inventory_Measure___SSA_(Public)',
       'Listings_Price_Cuts_(SA)_All_Homes',
       'Listings_Price_Cuts_(SA)_Condominiums',
       'Listings_Price_Cuts_(SA)_Single_Family_Residence',
       'Median_Listing_Price___All_Homes',
       'Median_Listing_Price___Condo/Co_op',
       'Median_Listing_Price___Duplex/Triplex',
       'Median_Listing_Price___Five_Or_More_Bedrooms',
       'Median_Listing_Price___Four_Bedrooms',
       'Median_Listing_Price___One_Bedroom',
       'Median_Listing_Price___Single_Family_Residence',
       'Median_Listing_Price___Three_Bedrooms',
       'Median_Listing_Price___Two_Bedrooms',
       'Median_Listing_Price_Per_Square_Foot___All_Homes',
       'Median_Listing_Price_Per_Square_Foot___Condo/C...',
       'Median_Listing_Price_Per_Square_Foot___Duplex/...',
       'Median_Listing_Price_Per_Square_Foot___Five_Or...',
       'Median_Listing_Price_Per_Square_Foot___

In [358]:
#Univariate feature selection
def univariate_feature_selector(data, k=7):
    uni_select = SelectKBest(chi2, k)
    uni_select.fit_transform(data,data.Median_Listing_Price___All_Homes)
    return data[data.columns[uni_select.get_support(indices=True)]]

sj_ds_uni = univariate_feature_selector(scaled_data, k=7) 


ValueError: Unknown label type: (array([0.19997852, 0.20145568, 0.19931339, 0.19812319, 0.19644199,
       0.19178938, 0.20266849, 0.2005442 , 0.20408771, 0.20551353,
       0.20562443, 0.20474303, 0.20291482, 0.20161395, 0.19949371,
       0.20256429, 0.20117106, 0.19648285, 0.19776623, 0.19716244,
       0.1965134 , 0.19839615, 0.20200869, 0.20291382, 0.20091459,
       0.1981514 , 0.19748704, 0.19804782, 0.19714322, 0.19426553,
       0.19400333, 0.20237649, 0.19891984, 0.19640519, 0.19978062,
       0.20046261, 0.2000054 , 0.19705177, 0.1990745 , 0.1972145 ,
       0.19810399, 0.19587599, 0.19670103, 0.19494361, 0.19815692,
       0.19860851, 0.19770092, 0.20369183, 0.19819613, 0.20337359,
       0.20421371, 0.2042607 , 0.20189663, 0.20062657, 0.19737342,
       0.19924391, 0.19980257, 0.20188569, 0.20171532, 0.20157238,
       0.19953043, 0.19805266, 0.20032292, 0.19850364, 0.20071245,
       0.20097183, 0.19849816, 0.1991079 , 0.19821813, 0.20097113,
       0.19850427, 0.198417  